# Trading Bot – Binance Futures Testnet  
**Candidate:** Gauri Nigam  
**Email:** nigamgauri24@gmail.com  

This project is a simplified trading bot built in Python using the Binance Futures Testnet. It was designed as part of a technical assignment to demonstrate basic automation of trading actions like placing market, limit, and stop-limit orders.

---

## What the Bot Does

- Connects to the Binance Futures Testnet using your API credentials  
- Lets you place **market**, **limit**, or **stop-limit** orders  
- Supports both **buy** and **sell** sides  
- Accepts user input via simple command-line prompts  
- Handles errors and logs order details automatically

---

## How to Use It (on Google Colab)

1. Run the first cell to install required libraries (`python-binance`)
2. Run the next cell to define the `BasicBot` class
3. Enter your Binance **Testnet API key and secret** (not your real one!)
4. Follow the command-line prompts to place an order:
   - Choose order type
   - Enter pair, side, quantity, and price (if needed)
5. You'll get order details in the output and logs in `bot.log`

---

This is a beginner-friendly trading bot, designed with clean structure, error handling, and logging — and leaves room for adding more advanced features like OCO, TWAP, or a basic UI.


In [1]:
!pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.6 MB/s eta 0:00:00


In [2]:
from binance.client import Client
import logging

class BasicBot:
    def __init__(self, api_key, api_secret, testnet=True):
        self.client = Client(api_key, api_secret, testnet=testnet)
        self.client.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'
        logging.basicConfig(filename='bot.log', level=logging.INFO)

    def place_market_order(self, symbol, side, quantity):
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type='MARKET',
                quantity=quantity
            )
            logging.info(f"Market Order: {order}")
            return order
        except Exception as e:
            logging.error(f"Market Order Error: {e}")
            return None

    def place_limit_order(self, symbol, side, quantity, price):
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type='LIMIT',
                timeInForce='GTC',
                quantity=quantity,
                price=price
            )
            logging.info(f"Limit Order: {order}")
            return order
        except Exception as e:
            logging.error(f"Limit Order Error: {e}")
            return None

    def place_stop_limit_order(self, symbol, side, quantity, stop_price, limit_price):
       try:
          order = self.client.futures_create_order(
             symbol=symbol,
             side=side,
             type='STOP',
             timeInForce='GTC',
             quantity=quantity,
             price=limit_price,
             stopPrice=stop_price,
             workingType='MARK_PRICE'
          )
          logging.info(f"Stop-Limit Order: {order}")
          return order
       except Exception as e:
          logging.error(f"Stop-Limit Order Error: {e}")
          return None


In [3]:
api_key = "1902fbb3738f9cbe4e1879f4cbf43e7fa178b07e855a7eb358b08849119d1270"
api_secret = "45fe0cc9a64778184ed2422bdf0910ad043046003078135b32810f7ab5a1a018"

bot = BasicBot(api_key, api_secret)

symbol = "BTCUSDT"
side = "BUY"
quantity = 0.001  # Small value for testing

result = bot.place_market_order(symbol, side, quantity)

print(result)


{'orderId': 5228886476, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'x-Cb7ytekJ5a81ed2bc9c1125ba0a334', 'price': '0.00', 'avgPrice': '0.00', 'origQty': '0.001', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1752038021188}


In [4]:
order_id = result['orderId']
status = bot.client.futures_get_order(symbol="BTCUSDT", orderId=order_id)
print(status)


{'orderId': 5228886476, 'symbol': 'BTCUSDT', 'status': 'FILLED', 'clientOrderId': 'x-Cb7ytekJ5a81ed2bc9c1125ba0a334', 'price': '0.00', 'avgPrice': '108491.80000', 'origQty': '0.001', 'executedQty': '0.001', 'cumQuote': '108.49180', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'time': 1752038021188, 'updateTime': 1752038021188}


In [5]:
symbol = "BTCUSDT"
side = "SELL"
quantity = 0.01
price = "12000"

order = bot.place_limit_order(symbol, side, quantity, price)
print(order)

ERROR:root:Limit Order Error: APIError(code=-4024): Limit price can't be lower than 54236.56.


None


In [6]:
symbol = "BTCUSDT"
side = "SELL"
quantity = 0.01
price = "54120"  # A little above the error limit

order = bot.place_limit_order(symbol, side, quantity, price)
print(order)

ERROR:root:Limit Order Error: APIError(code=-4024): Limit price can't be lower than 54236.60.


None


In [7]:
order = bot.place_stop_limit_order(
    symbol="BTCUSDT",
    side="SELL",
    quantity=0.01,
    stop_price="54000",
    limit_price="53990"   # Price to sell at
)
print(order)

{'orderId': 5228886905, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'x-Cb7ytekJ448b8903d0963d2da4741e', 'price': '53990.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '54000.00', 'workingType': 'MARK_PRICE', 'priceProtect': False, 'origType': 'STOP', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1752038025845}


In [8]:
order = bot.place_stop_limit_order(
    symbol="BTCUSDT",
    side="SELL",
    quantity=0.01,
    stop_price="54150",
    limit_price="54140"
)
print(order)

{'orderId': 5228886975, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'x-Cb7ytekJ6d195a11ab7cb9b8e93489', 'price': '54140.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '54150.00', 'workingType': 'MARK_PRICE', 'priceProtect': False, 'origType': 'STOP', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1752038026949}


In [13]:
order_type = input("Enter order type (MARKET / LIMIT / STOP_LIMIT): ").upper()
symbol = input("Enter trading pair (e.g., BTCUSDT): ").upper()
side = input("Enter side (BUY or SELL): ").upper()
quantity = float(input("Enter quantity: "))

if order_type == "MARKET":
    result = bot.place_market_order(symbol, side, quantity)

elif order_type == "LIMIT":
    price = input("Enter limit price: ")
    result = bot.place_limit_order(symbol, side, quantity, price)

elif order_type == "STOP_LIMIT":
    stop_price = input("Enter stop price: ")
    limit_price = input("Enter limit price: ")
    result = bot.place_stop_limit_order(symbol, side, quantity, stop_price, limit_price)

else:
    print("Invalid order type.")
    result = None

if result:
    print("Order placed successfully:")
    print(result)
else:
    print("Order failed. Check logs for details.")


Enter order type (MARKET / LIMIT / STOP_LIMIT): MARKET
Enter trading pair (e.g., BTCUSDT): BTCUSDT
Enter side (BUY or SELL): BUY
Enter quantity: 0.01
Order placed successfully:
{'orderId': 5226448730, 'symbol': 'BTCUSDT', 'status': 'NEW', 'clientOrderId': 'x-Cb7ytekJe57c115d3f71dd193a3089', 'price': '0.00', 'avgPrice': '0.00', 'origQty': '0.010', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1751999198829}
